# Necessary libraries

In [16]:
!pip install pmdarima

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.9 MB 7.3 MB/s            
     |████████████████████████████████| 3.6 MB 146.3 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [17]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import lagrange
from scipy.interpolate import CubicSpline, InterpolatedUnivariateSpline, Akima1DInterpolator
from datetime import datetime, timedelta
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import statsmodels.api as sm
#from datetime import timedelta
#import datetime
#import fbprophet
#from matplotlib import pyplot
from pandas import to_datetime
from statsmodels.tsa.arima_model import ARIMA
#from fbprophet import Prophet
#from pmdarima.arima import auto_arima
#import pmdarima
from scipy import interpolate
#from fbprophet import Prophet
from matplotlib import pyplot
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import plotly.graph_objs as go
import sklearn.model_selection
from scipy import stats
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow import keras
from imblearn.over_sampling import SMOTE
from scipy.interpolate import CubicSpline as CS
from statistics import mean
import math
from pmdarima.arima import auto_arima

In [ ]:
# df_for=pd.read_csv("6_df_interpolated_for_forecasting_may22.csv")
# df_for=df_for.drop(["Unnamed: 0"],axis=1)
# df_for

In [18]:
df_for=pd.read_csv("6_df_interpolated_for_forecasting_aug10.csv")
#df_for=df_for.drop(["Unnamed: 0"],axis=1)
df_for

,Unnamed: 0,Date,Area,Gender,Race,Ethinicity,Wound
0,0,2016-10-26,4.0000,M,White,Non-Hispanic,1
1,1,2016-10-27,4.0000,M,White,Non-Hispanic,1
2,2,2016-10-28,4.0000,M,White,Non-Hispanic,1
3,3,2016-10-29,4.0000,M,White,Non-Hispanic,1
4,4,2016-10-30,4.0000,M,White,Non-Hispanic,1
...,...,...,...,...,...,...,...
1290115,1290115,2023-01-05,2.7064,F,White,Non-Hispanic,17884
1290116,1290116,2023-01-06,2.7064,F,White,Non-Hispanic,17884
1290117,1290117,2023-01-07,2.7064,F,White,Non-Hispanic,17884
1290118,1290118,2023-01-08,2.7064,F,White,Non-Hispanic,17884


In [19]:
#df_for.to_csv("df_merged_for_forecasting_may01.csv")

In [20]:
# # #function for model input in ARIMA only
# def make_input_for_arima(dff):
#     dff_interpolated = dff
#     train_set_number=int(0.85*len(dff_interpolated))
#     test_set_number=len(dff_interpolated)-train_set_number

#     #Creating train and test set 
#     train=dff_interpolated[0:train_set_number] 
#     test=dff_interpolated[train_set_number:]

#     #to get the future dates for forecasting
#     date_specific=test.loc[:,'Date']
#     ff=date_specific.reset_index(drop=True)
#     #print(ff)
#     import datetime
#     for aa in range(0,14,1):    #range is given till 14 days as we are forecasting for 14 days or 2 weeks
#         u = datetime.datetime.strptime(str(ff[len(ff)-1]),"%Y-%m-%d")
#         d = datetime.timedelta(days=1)
#         t = u + d     #adds one new day with the dataframe
#         tt =t.strftime("%Y-%m-%d")   #creates that added new date from datetime object to string
#         dict={'Date':[tt]}
#         dff_2 = pd.DataFrame(dict)
#         test = test.append(dff_2, ignore_index = True)
#         date_specific=test.loc[:,'Date']
#         ff=date_specific.reset_index(drop=True)

#     #Aggregating the dataset at daily level
#     dff_interpolated.Timestamp = pd.to_datetime(dff_interpolated.Date,format='%Y-%m-%d %H:%M:%S') 
#     dff_interpolated.index = dff_interpolated.Timestamp 
#     dff_interpolated = dff_interpolated.resample('D').mean()
#     train.Timestamp = pd.to_datetime(train.Date,format='%Y-%m-%d %H:%M:%S') 
#     train.index = train.Timestamp 
#     train = train.resample('D').mean()
#     test.Timestamp = pd.to_datetime(test.Date,format='%Y-%m-%d %H:%M:%S') 
#     test.index = test.Timestamp 
#     test = test.resample('D').mean()
#     return (test, train)

In [21]:
def make_input_for_arima(df):    
    #display(df)
    df=df.reset_index()
    total_n=df.shape[0]
    n=math.ceil(0.15*total_n)
    dff=pd.concat([df, df.tail(n), df.tail(n)]).drop_duplicates(keep=False)
    #display(test)
    #display(dff)
    sample_row=dff.sample(1)
    a=int(sample_row.index.values)
    b=int(sample_row.index.values)+n
    #print(a)
    #print(b)
    test=df[a:b]
    train=pd.concat([df, test, test]).drop_duplicates(keep=False)
    # #display(train)
    return (test,train)

In [26]:
def model_holtwinters(test, train):
    #fit1 = sm.tsa.statespace.SARIMAX(train.Area, order=(2, 1, 4),seasonal_order=(0,1,1,7),initialization='approximate_diffuse').fit()
    fit1 = ExponentialSmoothing(np.asarray(train['Area']) ,seasonal_periods=2 ,trend='add', seasonal='add').fit()

    y_hat_avg = test.copy()
    y_hat_avg['Holtwinters'] = fit1.forecast(len(test))
    #print(y_hat_avg['ARIMA'])
    plt.figure(figsize=(16,8))
    plt.plot( train['Area'], label='Train')
    plt.plot(test['Area'], label='Test')
    plt.plot(y_hat_avg['Holtwinters'], label='Holtwinters')
    plt.legend(loc='best')
    #plt.axhline(0)
    plt.grid()
    plt.xlabel('Date')
    plt.ylabel('Wound Area')
    plt.show()




    #Accuracy measure of the model (to be done next)
    forecast=y_hat_avg.Holtwinters
    actual=test.Area
    training_series=train.Area
    testing_series=test.Area
    prediction_series= y_hat_avg.Holtwinters



    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
     #as R-square value does not for forecasted value and actual value because forecasted values are 14 more than actual values, thus I am taking only the forecasted values minusing the last 14 values of the forecasted dataframe.
    # df = df.iloc[:-2]
    #forecast_without_last14=forecast.iloc[:-14]
    #actual_without_nan=actual.dropna()
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    #corr= np.corrcoef(forecast_without_last14,actual_without_nan)[0,1]
    r_square= corr**2
#     mins = np.amin(np.hstack([forecast[:,None], 
#                               actual[:,None]]), axis=1)
#     maxs = np.amax(np.hstack([forecast[:,None], 
#                               actual[:,None]]), axis=1)
#     minmax = 1 - np.mean(mins/maxs)             # minmax
    #acf1 = acf(fc-test)[1]  # ACF1

    n = training_series.shape[0]
    d = np.abs(  np.diff( training_series) ).sum()/(n-1)
    errors = np.abs(testing_series - prediction_series )
    mase=errors.mean()/d

    #print("For Subject ID:", list[i])
#     print("MASE:",mase)
#     print("MAPE:",mape)
#     print("ME:",me)
#     print("MAE:",mae)
#     print("MPE:",mpe)
#     print("RMSE:",rmse)
#     print("CORR:",corr)
#     print("R-SQUARE",r_square)
#     print("MINMAX:",minmax)
    return (rmse, mase, r_square)




## Counting the number of wounds from patients from different categories and running ARIMA model

In [27]:
# df_for_f=df_for.query("Gender=='F'")
# df_for_m=df_for.query("Gender=='M'")

# df_for_White=df_for.query("Race=='White'")
# df_for_Black=df_for.query("Race=='Black'")
# df_for_ASKU=df_for.query("Race=='ASKU'")
# df_for_Asian=df_for.query("Race=='Asian'")
# df_for_American Indian or Alaska Native=df_for.query("Race=='American Indian or Alaska Native'")
# df_for_Native Hawaiian or Other Pacific Islander=df_for.query("Race=='Native Hawaiian or Other Pacific Islander'")
# df_for_Other=df_for.query("Race=='Other'")

In [28]:
# list_female_r_square=[]
# list_male_r_square=[]
# list_white_r_square=[]
# list_black_r_square=[]
# list_asku_r_square=[]
# list_asian_r_square=[]
# list_americanindian_r_square=[]
# list_nativehawaiian_r_square=[]
# list_other_r_square=[]
# list_hispanic_r_square=[]
# list_nonhispanic_r_square=[]
# list_patientdeclined_r_square=[]


# list_female_rmse=[]
# list_male_rmse=[]
# list_white_rmse=[]
# list_black_rmse=[]
# list_asku_rmse=[]
# list_asian_rmse=[]
# list_americanindian_rmse=[]
# list_nativehawaiian_rmse=[]
# list_other_rmse=[]
# list_hispanic_rmse=[]
# list_nonhispanic_rmse=[]
# list_patientdeclined_rmse=[]


# list_female_mase=[]
# list_male_mase=[]
# list_white_mase=[]
# list_black_mase=[]
# list_asku_mase=[]
# list_asian_mase=[]
# list_americanindian_mase=[]
# list_nativehawaiian_mase=[]
# list_other_mase=[]
# list_hispanic_mase=[]
# list_nonhispanic_mase=[]
# list_patientdeclined_mase=[]


# list_female_nan=0
# list_male_nan=0
# list_white_nan=0
# list_black_nan=0
# list_asku_nan=0
# list_asian_nan=0
# list_americanindian_nan=0
# list_nativehawaiian_nan=0
# list_other_nan=0
# list_hispanic_nan=0
# list_nonhispanic_nan=0
# list_patientdeclined_nan=0


# list_wounds=df_for["Wound"].unique()

# for i in range(len(list_wounds)):
# #     if i>4:
# #         break
#     print("Wound:",i+1)
#     wound=list_wounds[i]
#     df_1=df_for.query("Wound==@wound")
#     #display(df)
#     df=df_1[["Date","Area"]]
#     test,train=make_input_for_arima(df)
#     if train.shape[0]>1:
#         if train.shape[0]==2 or train.shape[0]==3:
#             if len(list(set(train.Area.values.tolist())))>1 & (datetime.strptime(train.reset_index()["Date"][train.shape[0]-1],'%Y-%m-%d')-datetime.strptime(train.reset_index()["Date"][0],'%Y-%m-%d')).days>2 and train["Area"].dropna().shape[0]!=0:
            
#                 rmse, mase, r_square=model_holtwinters(test, train)

#                 if str(r_square)=='nan' or str(r_square)=='inf':

#                     #Check gender
#                     if df_1["Gender"].unique()[0]=='F':
#                         list_female_nan=list_female_nan+1
#                     elif df_1["Gender"].unique()[0]=='M':
#                         list_male_nan=list_male_nan+1

#                     #Check race
#                     if df_1["Race"].unique()[0]=='White':
#                         list_white_nan=list_white_nan+1
#                     elif df_1["Race"].unique()[0]=='Black':
#                         list_black_nan=list_black_nan+1
#                     if df_1["Race"].unique()[0]=='ASKU':
#                         list_asku_nan=list_asku_nan+1
#                     if df_1["Race"].unique()[0]=='Asian':
#                         list_asian_nan=list_asian_nan+1
#                     if df_1["Race"].unique()[0]=='American Indian or Alaska Native':
#                         list_americanindian_nan=list_americanindian_nan+1
#                     if df_1["Race"].unique()[0]=='Native Hawaiian or Other Pacific Islander':
#                         list_nativehawaiian_nan=list_nativehawaiian_nan+1
#                     if df_1["Race"].unique()[0]=='Other':
#                         list_other_nan=list_other_nan+1

#                     #Check ethnicity
#                     if df_1["Ethinicity"].unique()[0]=='Hispanic':
#                         list_hispanic_nan=list_hispanic_nan+1
#                     elif df_1["Ethinicity"].unique()[0]=='Non-Hispanic':
#                         list_nonhispanic_nan=list_nonhispanic_nan+1
#                     if df_1["Ethinicity"].unique()[0]=='Patient Declined':
#                         list_patientdeclined_nan=list_patientdeclined_nan+1

#                 else:

#                     #Check gender
#                     if df_1["Gender"].unique()[0]=='F':
#                         list_female_r_square.append(r_square)
#                         list_female_rmse.append(rmse)
#                         list_female_mase.append(mase)

#                     elif df_1["Gender"].unique()[0]=='M':
#                         list_male_r_square.append(r_square)
#                         list_male_rmse.append(rmse)
#                         list_male_mase.append(mase)


#                     #Check race
#                     if df_1["Race"].unique()[0]=='White':
#                         list_white_r_square.append(r_square)
#                         list_white_rmse.append(rmse)
#                         list_white_mase.append(mase)
#                     elif df_1["Race"].unique()[0]=='Black':
#                         list_black_r_square.append(r_square)
#                         list_black_rmse.append(rmse)
#                         list_black_mase.append(mase)
#                     if df_1["Race"].unique()[0]=='ASKU':
#                         list_asku_r_square.append(r_square)
#                         list_asku_rmse.append(rmse)
#                         list_asku_mase.append(mase)
#                     if df_1["Race"].unique()[0]=='Asian':
#                         list_asian_r_square.append(r_square)
#                         list_asian_rmse.append(rmse)
#                         list_asian_mase.append(mase)
#                     if df_1["Race"].unique()[0]=='American Indian or Alaska Native':
#                         list_americanindian_r_square.append(r_square)
#                         list_americanindian_rmse.append(rmse)
#                         list_americanindian_mase.append(mase)
#                     if df_1["Race"].unique()[0]=='Native Hawaiian or Other Pacific Islander':
#                         list_nativehawaiian_r_square.append(r_square)
#                         list_nativehawaiian_rmse.append(rmse)
#                         list_nativehawaiian_mase.append(mase)
#                     if df_1["Race"].unique()[0]=='Other':
#                         list_other_r_square.append(r_square)
#                         list_other_rmse.append(rmse)
#                         list_other_mase.append(mase)

#                     #Check ethnicity
#                     if df_1["Ethinicity"].unique()[0]=='Hispanic':
#                         list_hispanic_r_square.append(r_square)
#                         list_hispanic_rmse.append(rmse)
#                         list_hispanic_mase.append(mase)
#                     elif df_1["Ethinicity"].unique()[0]=='Non-Hispanic':
#                         list_nonhispanic_r_square.append(r_square)
#                         list_nonhispanic_rmse.append(rmse)
#                         list_nonhispanic_mase.append(mase)
#                     if df_1["Ethinicity"].unique()[0]=='Patient Declined':
#                         list_patientdeclined_r_square.append(r_square)
#                         list_patientdeclined_rmse.append(rmse)
#                         list_patientdeclined_mase.append(mase)
                    
#         if train.shape[0]>3 and train["Area"].dropna().shape[0]!=0:
            
#             rmse, mase, r_square=model_holtwinters(test, train)

#             if str(r_square)=='nan' or str(r_square)=='inf':

#                 #Check gender
#                 if df_1["Gender"].unique()[0]=='F':
#                     list_female_nan=list_female_nan+1
#                 elif df_1["Gender"].unique()[0]=='M':
#                     list_male_nan=list_male_nan+1

#                 #Check race
#                 if df_1["Race"].unique()[0]=='White':
#                     list_white_nan=list_white_nan+1
#                 elif df_1["Race"].unique()[0]=='Black':
#                     list_black_nan=list_black_nan+1
#                 if df_1["Race"].unique()[0]=='ASKU':
#                     list_asku_nan=list_asku_nan+1
#                 if df_1["Race"].unique()[0]=='Asian':
#                     list_asian_nan=list_asian_nan+1
#                 if df_1["Race"].unique()[0]=='American Indian or Alaska Native':
#                     list_americanindian_nan=list_americanindian_nan+1
#                 if df_1["Race"].unique()[0]=='Native Hawaiian or Other Pacific Islander':
#                     list_nativehawaiian_nan=list_nativehawaiian_nan+1
#                 if df_1["Race"].unique()[0]=='Other':
#                     list_other_nan=list_other_nan+1

#                 #Check ethnicity
#                 if df_1["Ethinicity"].unique()[0]=='Hispanic':
#                     list_hispanic_nan=list_hispanic_nan+1
#                 elif df_1["Ethinicity"].unique()[0]=='Non-Hispanic':
#                     list_nonhispanic_nan=list_nonhispanic_nan+1
#                 if df_1["Ethinicity"].unique()[0]=='Patient Declined':
#                     list_patientdeclined_nan=list_patientdeclined_nan+1

#             else:

#                 #Check gender
#                 if df_1["Gender"].unique()[0]=='F':
#                     list_female_r_square.append(r_square)
#                     list_female_rmse.append(rmse)
#                     list_female_mase.append(mase)

#                 elif df_1["Gender"].unique()[0]=='M':
#                     list_male_r_square.append(r_square)
#                     list_male_rmse.append(rmse)
#                     list_male_mase.append(mase)


#                 #Check race
#                 if df_1["Race"].unique()[0]=='White':
#                     list_white_r_square.append(r_square)
#                     list_white_rmse.append(rmse)
#                     list_white_mase.append(mase)
#                 elif df_1["Race"].unique()[0]=='Black':
#                     list_black_r_square.append(r_square)
#                     list_black_rmse.append(rmse)
#                     list_black_mase.append(mase)
#                 if df_1["Race"].unique()[0]=='ASKU':
#                     list_asku_r_square.append(r_square)
#                     list_asku_rmse.append(rmse)
#                     list_asku_mase.append(mase)
#                 if df_1["Race"].unique()[0]=='Asian':
#                     list_asian_r_square.append(r_square)
#                     list_asian_rmse.append(rmse)
#                     list_asian_mase.append(mase)
#                 if df_1["Race"].unique()[0]=='American Indian or Alaska Native':
#                     list_americanindian_r_square.append(r_square)
#                     list_americanindian_rmse.append(rmse)
#                     list_americanindian_mase.append(mase)
#                 if df_1["Race"].unique()[0]=='Native Hawaiian or Other Pacific Islander':
#                     list_nativehawaiian_r_square.append(r_square)
#                     list_nativehawaiian_rmse.append(rmse)
#                     list_nativehawaiian_mase.append(mase)
#                 if df_1["Race"].unique()[0]=='Other':
#                     list_other_r_square.append(r_square)
#                     list_other_rmse.append(rmse)
#                     list_other_mase.append(mase)

#                 #Check ethnicity
#                 if df_1["Ethinicity"].unique()[0]=='Hispanic':
#                     list_hispanic_r_square.append(r_square)
#                     list_hispanic_rmse.append(rmse)
#                     list_hispanic_mase.append(mase)
#                 elif df_1["Ethinicity"].unique()[0]=='Non-Hispanic':
#                     list_nonhispanic_r_square.append(r_square)
#                     list_nonhispanic_rmse.append(rmse)
#                     list_nonhispanic_mase.append(mase)
#                 if df_1["Ethinicity"].unique()[0]=='Patient Declined':
#                     list_patientdeclined_r_square.append(r_square)
#                     list_patientdeclined_rmse.append(rmse)
#                     list_patientdeclined_mase.append(mase)

In [ ]:
list_wounds=df_for["Wound"].unique()

result_df=pd.DataFrame(index=np.arange(len(list_wounds)))

result_df["Wound"]=np.nan
result_df["Gender"]=np.nan
result_df["Race"]=np.nan
result_df["Ethinicity"]=np.nan
result_df["RMSE"]=np.nan
result_df["MASE"]=np.nan
result_df["R-squared"]=np.nan
    

for i in range(len(list_wounds)):
#     if i>4:
#         break
    print("Wound:",i+1)
    wound=list_wounds[i]
    df_1=df_for.query("Wound==@wound")
    #display(df_1)
    df=df_1[["Date","Area"]]
    test,train=make_input_for_arima(df)
    
    
    result_df["Wound"][i]=int(df_1["Wound"].unique()[0])
    result_df["Gender"][i]=str(df_1["Gender"].unique()[0])
    result_df["Race"][i]=str(df_1["Race"].unique()[0])
    result_df["Ethinicity"][i]=str(df_1["Ethinicity"].unique()[0])

    if train.shape[0]>1:
        if train.shape[0]==2 or train.shape[0]==3:
            if len(list(set(train.Area.values.tolist())))>1 & (datetime.strptime(train.reset_index()["Date"][train.shape[0]-1],'%Y-%m-%d')-datetime.strptime(train.reset_index()["Date"][0],'%Y-%m-%d')).days>2 and train["Area"].dropna().shape[0]!=0:
            
                rmse, mase, r_square=model_holtwinters(test, train)
                result_df["RMSE"][i]=rmse
                result_df["MASE"][i]=mase
                result_df["R-squared"][i]=r_square
                
        if train.shape[0]>3 and train["Area"].dropna().shape[0]!=0:
            
            rmse, mase, r_square=model_holtwinters(test, train)
            result_df["RMSE"][i]=rmse
            result_df["MASE"][i]=mase
            result_df["R-squared"][i]=r_square
                
result_df["Wound"]=result_df["Wound"].astype(int)

In [30]:
result_df

,Wound,Gender,Race,Ethinicity,RMSE,MASE,R-squared
0,1,M,White,Non-Hispanic,1.479525,5.066716,0.046923
1,2,M,White,Non-Hispanic,1.014797,4.723285,0.927650
2,3,M,White,Non-Hispanic,0.185481,7.490611,1.000000
3,5,M,White,Non-Hispanic,2.328319,6.992540,0.993764
4,6,M,White,Non-Hispanic,0.872019,15.624282,0.979899
...,...,...,...,...,...,...,...
14913,17877,M,White,Non-Hispanic,2.596683,16.561704,0.887877
14914,17879,F,White,Non-Hispanic,0.093146,5.265298,NaN
14915,17880,F,White,Non-Hispanic,1.795225,5.114129,0.953891
14916,17881,M,White,Non-Hispanic,13.141264,4.433311,1.000000


In [34]:
result_df.to_csv("Holt_winters_result_Aug24.csv")

In [32]:
result_df=result_df.dropna()
result_df=result_df.replace({np.inf:np.nan})
result_df=result_df.dropna()

In [33]:
result_df

,Wound,Gender,Race,Ethinicity,RMSE,MASE,R-squared
0,1,M,White,Non-Hispanic,1.479525,5.066716,0.046923
1,2,M,White,Non-Hispanic,1.014797,4.723285,0.927650
2,3,M,White,Non-Hispanic,0.185481,7.490611,1.000000
3,5,M,White,Non-Hispanic,2.328319,6.992540,0.993764
4,6,M,White,Non-Hispanic,0.872019,15.624282,0.979899
...,...,...,...,...,...,...,...
14911,17875,F,White,Non-Hispanic,0.378447,8.993306,0.996527
14912,17876,F,White,Non-Hispanic,6.212400,9.657690,0.930276
14913,17877,M,White,Non-Hispanic,2.596683,16.561704,0.887877
14915,17880,F,White,Non-Hispanic,1.795225,5.114129,0.953891
